## Building Blocks of DSPy

- **dspy.Signature**: define the input/output contract of DSPy module.
- **dspy.Module**: define the logic of interacting with LLMs.

## Advantages

- LM-agnostic programming
- Seamless productization
- Automatic program optimization

In [ ]:
import dspy
import mlflow
from typing import Literal
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"), track_usage=True)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-local-experiment")
mlflow.dspy.autolog()

# running local MLflow server
# mlflow server --host 127.0.0.1 --port 5000

## Class-based Signature Example

In [ ]:
class SlangEquivalence(dspy.Signature):
    """Find a Portuguese slang equivalance in English and classify it's formality level."""

    # inputs
    portuguese_slang: str = dspy.InputField()
    # outputs
    english_slang: str = dspy.OutputField()
    alternative_slangs: list[str] = dspy.OutputField(
        desc="More equivalent alternative slangs."
    )
    formality_level: Literal["L1", "L2", "L3", "L4", "L5"] = dspy.OutputField(
        desc="Slang formality level. Lesser the number more informal."
    )

## String-based Signature Example

In [ ]:
slang_equivalance = dspy.make_signature(
    "portuguese_slang -> english_slang: str, alternative_slangs: list[str]"
)

## LM Interaction via Module Examples

In [ ]:
predict = dspy.Predict(slang_equivalance)
output = predict(portuguese_slang="Boiar")

In [ ]:
output

In [ ]:
print(
    f"English equivalent slang: {output['english_slang']}\nAlternative slangs: {output.alternative_slangs}"
)

In [ ]:
cot = dspy.ChainOfThought(SlangEquivalence)
output = cot(portuguese_slang="quebrar o galho")

In [ ]:
output

In [ ]:
dspy.inspect_history(n=1)

## Changing the Adapter

In [ ]:
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o"))
dspy.configure(adapter=dspy.JSONAdapter())

In [ ]:
cot = dspy.ChainOfThought(SlangEquivalence)
output = cot(portuguese_slang="quebrar o galho")

In [ ]:
output

In [ ]:
dspy.inspect_history(n=1)

## Creating a Custom Module

In [ ]:
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"), track_usage=False)

In [ ]:
class SkillIdentifier(dspy.Signature):
    """Given a job description, identify the most important technical skills needed for a successfully job appliance."""

    job_description: str = dspy.InputField()
    tech_skills: list[str] = dspy.OutputField(
        desc="Ordered technical skills from most important to less important"
    )


class SkillSummary(dspy.Signature):
    """Given a technical skill make a short summary showing what is it about and give some examples during the explanation. Also include examples of project based ideas to prove the knowledge about the skill."""

    tech_skill: str = dspy.InputField()
    skill_summary: str = dspy.OutputField(
        desc="A Markdown-formatted summary section"
    )
    project_based_examples: list[str] = dspy.OutputField()


class DetailJob(dspy.Module):
    def __init__(self):
        self.skill_identifier = dspy.Predict(SkillIdentifier)
        self.skill_summary = dspy.Predict(SkillSummary)

    def forward(self, job_description, top_k=3):
        skills_resp = self.skill_identifier(job_description=job_description)
        print(f"## IDENTIFIED SKILLS: {skills_resp.tech_skills}")
        for i, item in enumerate(skills_resp.tech_skills):
            details_resp = self.skill_summary(tech_skill=item)

            print("=" * 50)
            print(f"* SKILL: {item}")
            print("=" * 50)
            print(f">> SUMMARY:\n{details_resp.skill_summary}")
            print(
                f"---\n>> PROJECT BASED IDEAS:\n{details_resp.project_based_examples}"
            )

            if i == top_k - 1:
                break

In [ ]:
description = """
About the Role As an AI Research Engineer, you will: Build and maintain LLM-based and agentic product features, including
rigorous evaluation, benchmarking and backtesting. Create systems that can improve over time based on feedback from humans
in the loop, such as prompt optimisation and fine-tuning.s Explore applications of the latest developments of AI to enhance
our product features, explainability, speed and accuracy. What You’ll Do AI features: Use the latest in LLMs, agents and
computer vision to build systems that are faster, more thorough and more accurate than humans. Evals: Build benchmarks,
evaluate backtests and analyse results to quantify our performance and prevent regressions. MLOps: Ensure that we maintain
good practices around collecting and managing datasets, backtests and benchmarks to to maximise development velocity Prompt
Engineering: Develop and test prompts to optimise model performance and deliver high-quality outputs. Model Fine-Tuning:
Adapt and fine-tune large language models and vision models for specific use cases. Custom Model Training: Train and deploy
bespoke AI models tailored to solve unique compliance challenges. Collaboration: Work closely with the engineering and
product teams to translate customer needs into actionable AI solutions. Iterate: Continuously iterate on our AI systems to
improve performance, reduce errors, and deliver value to customers. Our Culture 1. Own With Urgency As a startup, delivering
value quickly is our superpower. We achieve this by valuing speed and ownership over perfection. We're not afraid to get our
hands dirty, experiment, and iterate quickly to achieve our goals, and completely own the outcome. 2. Transparency We believe
in open communication and full visibility across teams and roles. Decisions, successes, and failures are shared openly to
foster trust and collaboration. 3. Customer First, Team Second, Self Last Our priority is creating value for our customers.
We then focus on building a supportive, growth-oriented team environment, putting individual needs last to ensure collective
success. What We’re Looking For Experience: 3+ years in an AI research or engineering role, with experience building and
testing agentic systems. Technical Expertise: Hands-on experience with prompt engineering, fine-tuning pre-trained models,
and training custom models, including vision models. Experience using TypeScript a plus. Product Mindset: Strong
understanding of how AI can solve real-world problems, with a focus on customer needs. Interests: Ability to stay updated with
the latest advancements in AI and apply them effectively. Collaboration: Excellent communication skills to work across teams
and explain complex AI concepts to non-technical stakeholders. Ownership: A proactive, problem-solving mindset with the
ability to take full responsibility for projects and outcomes. Growth-Oriented: Excited to learn new skills, tackle challenges,
and adapt as the company scales.
"""

In [ ]:
job_detail = DetailJob()
job_detail(job_description=description)

## Saving and Loading

In [ ]:
job_detail.save("data/dspy_modules/jobdetail.json", save_program=False)

In [ ]:
job_detail.load("data/dspy_modules/jobdetail.json")

In [ ]:
job_detail.save("data/dspy_modules/jobdetail/", save_program=True)

In [ ]:
loaded_module = dspy.load("data/dspy_modules/jobdetail/")

In [ ]:
# loaded_module()